<a href="https://colab.research.google.com/github/Utkarsh13tiwari/Deep-Learning-Models/blob/main/EfficientNet_Classificationipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip


--2023-08-31 21:06:18--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 142.250.4.128, 74.125.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  20.2MB/s    in 3.9s    

2023-08-31 21:06:23 (16.7 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
import zipfile
import os
import tensorflow
from tensorflow import keras
from keras import layers
from keras.layers import Flatten,Dense,Dropout
from tensorflow.keras.optimizers import RMSprop
from keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory( validation_dir,  batch_size = 20, class_mode = 'binary', target_size = (224, 224))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
from keras.applications import EfficientNetB0

In [ ]:
base_model = EfficientNetB0(input_shape=(224,224,3),include_top=False,weights='imagenet')

16705208/16705208 [==============================] - 2s 0us/step


In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024,activation="relu")(x)
x = Dropout(0.5)(x)

predictions = Dense(1,activation = 'sigmoid')(x)
model_final = Model(base_model.input , predictions)

In [ ]:
model_final.compile(optimizer=tensorflow.keras.optimizers.legacy.RMSprop(lr=0.0001, decay=1e-6),loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
eff_history = model_final.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)

<ipython-input-11-5e73dccce911>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  eff_history = model_final.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)


Epoch 1/10
100/100 [==============================] - 59s 437ms/step - loss: 3.3068 - accuracy: 0.4870 - val_loss: 1.3222 - val_accuracy: 0.5000
Epoch 2/10
100/100 [==============================] - 37s 374ms/step - loss: 0.9988 - accuracy: 0.5105 - val_loss: 0.6947 - val_accuracy: 0.5000
Epoch 3/10
100/100 [==============================] - 38s 382ms/step - loss: 0.7071 - accuracy: 0.4970 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 4/10
100/100 [==============================] - 35s 355ms/step - loss: 0.6943 - accuracy: 0.4950 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/10
100/100 [==============================] - 36s 356ms/step - loss: 0.6953 - accuracy: 0.4890 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/10
100/100 [==============================] - 37s 369ms/step - loss: 0.6952 - accuracy: 0.5115 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/10
100/100 [==============================] - 35s 353ms/step - loss: 0.6941 - accuracy: 0.5090 - val_loss: 0.6932 - val_ac